<a href="https://colab.research.google.com/github/arnaldog12/Machine_Learning/blob/master/Naive_Bayes_Gaussian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

|  |  |
|-------------|-------|
| 🎓 **Aprendizado** | Supervisionado |
| 📋 **Tarefa** | Classificação |
| 🔧 **Normalização** | Sim |
| ⭐ **Dificuldade** | Fácil |

# ⚙️ 0. Dependências

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# 🔍 1. Introdução

**Naive Bayes**, técnicamente conhecido como *Posterior Probability*, é um técnica simples de construção de classificadores. Não é um único algoritmo para treinamento de classificadores, mas uma família de algoritmos baseada num princípio em comum: todos os classificadores Naive Bayes assumem que o valor de um atributo em particular é *independente* do valor dos outros atributos, dada a variável da classe.

Isso é o que torna o Naive Bayes *"Naive"*: ele não considera a dependência entre os atributos. Se pensarmos no caso de classificação textual - onde cada palavra representa um atributo, por exemplo - a ordem das palavras são importantes para a classificação.

✅ **Vantagens**:
- Simples de implementar
- Funciona bem com poucos dados de treinamento
- Não é sensível a características irrelevantes
- Facilmente **escalável**
- Bom desempenho em classificação de texto

❌ **Desvantagens**:
- Suposição de independência raramente é verdadeira (mesmo assim funciona bem em muitos casos)
- Pode ser superado por outros classificadores, como árvores de decisão e Random Forests.

## Teorema de Bayes

A famosa equação de Bayes nos permite fazer predições a partir dos dados. Tal equação é definida pela seguinte fórmula:

$$
P(A\mid B)={\frac {P(B\mid A)\,P(A)}{P(B)}}
$$

Para torná-la menos abstrata, vamos substituir as variáveis $A$ e $B$ por nomes mais tangíveis. Dessa forma, é melhor pensar na equação acima da seguinte forma:

<img src="https://github.com/arnaldog12/Machine_Learning/blob/master/images/teorema-bayes.jpg?raw=1" width="600px">

[Fonte](https://github.com/odubno/GaussNaiveBayes)

Onde:
- **Posterior Probability**: essa é resposta da predição do nosso Naive Bayes para uma nova amostra, onde cada valor representa a probabilidade da amostra pertencer a cada classe.
- **Class Prior Probability**: a probabilidade a priori de uma determinada classe.
- **Likelihood**: a verossimilhança é calculada pelo produtório de todas as *Funções de Densidade de Probabilidade Normal* (**Normal Probability Density Functions**). A FDP Normal é calculada usando a distribuição Gaussiana. Daí o nome Gaussian Naive Bayes. Nós utilizaremos a FDP Normal para calcular o valor da probabilidade normal para cada atributo dado uma classe. A FDP Normal é dada pela seguinte fórmula:

$$
f(x \mid \mu,\sigma^2) = \frac{1}{\sqrt{2\pi \sigma^2}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}
$$

> **⚠️ É importante não confundir verossimilhança com probabilidade**:
- A **probabilidade** é calculada para cada atributo dada uma classe e seu valor está sempre entre 0 e 1.
- A **verossimilhança** é o produto de todos os valores de probabilidade normal. <br>
👀 Veja [esse link](https://github.com/odubno/GaussNaiveBayes) para entender como a *likelihood* é calculada.

- **Predictor Prior Probability**: é o mesmo que **Probabilidade Marginal** (*Marginal Probability*). Representa a probabilidade dados os novos dados sob todos os valores possíveis de atributos para cada classe. Repare que não é necessário calcular esse valor (eles apenas normaliza as nossas probabilidades). Sem esse termo, temos as predições. Com ele, nós temos a probabilidade exata. Entretanto, não normalizar as predições (para gerar as probabilidades) não altera o resultado final.

# 🎲 2. Dados

In [2]:
iris = load_iris()

df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df["class"] = iris.target
df.sample(10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
101,5.8,2.7,5.1,1.9,2
57,4.9,2.4,3.3,1.0,1
45,4.8,3.0,1.4,0.3,0
30,4.8,3.1,1.6,0.2,0
2,4.7,3.2,1.3,0.2,0
20,5.4,3.4,1.7,0.2,0
75,6.6,3.0,4.4,1.4,1
127,6.1,3.0,4.9,1.8,2
125,7.2,3.2,6.0,1.8,2
12,4.8,3.0,1.4,0.1,0


In [3]:
x = df.drop(labels="class", axis=1).values
y = df["class"].values

print(x.shape, y.shape)

(150, 4) (150,)


# 💻 3. Implementação

## 🏷️ Classificador (features numéricas)

- `theta`: $\mu$
- `var`: $\sigma^2$
- `normal_pdf`: $f(x \mid \mu,\sigma^2) = P({features} \mid {class})$


- `priors`: $P(class)$ ➡️ class prior probability
- `joint_prob`: $P(class) * P({features} \mid {class})$


In [4]:
class GaussianNaiveBayes:
    def __init__(self, priors=None):
        self.priors = priors
        self.theta_ = 0.0
        self.var_ = 0.0

    def fit(self, x, y):
        classes, counts = np.unique(y, return_counts=True)
        self.priors = self.priors or counts / counts.sum()
        self.theta_ = np.array([np.mean(x[y == c], axis=0) for c in classes])
        self.var_ = np.array([np.var(x[y == c], axis=0) for c in classes])

    def predict(self, x):
        return np.argmax(self.predict_proba(x), axis=1)

    def predict_proba(self, x):
        y_pred = []
        for sample in x:
            joint_prob = self.__joint_prob(sample)
            marginal_prob = np.sum(joint_prob)
            posterior_prob = joint_prob / marginal_prob
            y_pred.append(posterior_prob)
        return np.array(y_pred)

    def __joint_prob(self, x):
        joint_prob = []
        for p, t, s in zip(self.priors, self.theta_, self.var_):
            joint_prob.append(p * self.__normal_pdf(x, t, s))
        return joint_prob

    def __normal_pdf(self, x, mean_c, var_c):
        exponent = ((x - mean_c) ** 2) / (2 * var_c)
        f = (1.0 / np.sqrt(2.0 * np.pi * var_c)) * np.exp(-exponent)
        return np.prod(f)

In [5]:
clf = GaussianNaiveBayes(priors=[0.1, 0.2, 0.7])
clf.fit(x, y)

print(clf.theta_)
print(clf.var_)
print(clf.predict(x[::15]))
print(clf.predict_proba(x[::15]))

[[5.006 3.428 1.462 0.246]
 [5.936 2.77  4.26  1.326]
 [6.588 2.974 5.552 2.026]]
[[0.121764 0.140816 0.029556 0.010884]
 [0.261104 0.0965   0.2164   0.038324]
 [0.396256 0.101924 0.298496 0.073924]]
[0 0 0 0 1 1 1 2 2 2]
[[1.00000000e+000 2.71568036e-018 4.97897739e-025]
 [1.00000000e+000 6.08147999e-017 1.16347854e-022]
 [1.00000000e+000 2.15419961e-016 1.67026397e-023]
 [1.00000000e+000 3.90567500e-016 1.38142807e-023]
 [5.37488397e-042 9.99999177e-001 8.22738054e-007]
 [1.60315864e-094 9.57480597e-001 4.25194033e-002]
 [5.65738810e-083 9.99558019e-001 4.41981441e-004]
 [6.65703592e-274 6.88505128e-011 1.00000000e+000]
 [1.12727751e-222 3.34619351e-009 9.99999997e-001]
 [8.68653305e-255 2.56736467e-011 1.00000000e+000]]


In [6]:
y_pred = clf.predict(x)
print(accuracy_score(y, y_pred))

0.9533333333333334


### Comparação com o Scikit-learn

In [7]:
clf_sk = GaussianNB(priors=[0.1, 0.2, 0.7], var_smoothing=0.0)
clf_sk.fit(x, y)

print(clf_sk.theta_)
print(clf_sk.var_)
print(clf_sk.predict(x[::15]))
print(clf_sk.predict_proba(x[::15]))

[[5.006 3.428 1.462 0.246]
 [5.936 2.77  4.26  1.326]
 [6.588 2.974 5.552 2.026]]
[[0.121764 0.140816 0.029556 0.010884]
 [0.261104 0.0965   0.2164   0.038324]
 [0.396256 0.101924 0.298496 0.073924]]
[0 0 0 0 1 1 1 2 2 2]
[[1.00000000e+000 2.71568036e-018 4.97897739e-025]
 [1.00000000e+000 6.08147999e-017 1.16347854e-022]
 [1.00000000e+000 2.15419961e-016 1.67026397e-023]
 [1.00000000e+000 3.90567500e-016 1.38142807e-023]
 [5.37488397e-042 9.99999177e-001 8.22738054e-007]
 [1.60315864e-094 9.57480597e-001 4.25194033e-002]
 [5.65738810e-083 9.99558019e-001 4.41981441e-004]
 [6.65703592e-274 6.88505128e-011 1.00000000e+000]
 [1.12727751e-222 3.34619351e-009 9.99999997e-001]
 [8.68653305e-255 2.56736467e-011 1.00000000e+000]]


In [8]:
y_pred = clf_sk.predict(x)
print(accuracy_score(y, y_pred))

0.9533333333333334


# 💭 Considerações Finais

## Variações do Naive Bayes

- **Multinomial Naive Bayes**: usado para dados discretos (e.g., contagem de palavras em classificação de texto).

- **Bernoulli Naive Bayes**: para features binárias (e.g., presença/ausência de palavras).

# 📖 Referências

- [Repositório do GitHub](https://github.com/odubno/GaussNaiveBayes)
- [Naive Bayes Classifier From Scratch](https://chrisalbon.com/machine_learning/naive_bayes/naive_bayes_classifier_from_scratch/)